In [1]:
#https://sbert.net/
#https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
#https://huggingface.co/openai/whisper-large-v3
#https://pypi.org/project/faiss-cpu/

In [2]:
import json

# Load the list from the JSON file
with open('speech_total.json', 'r') as file:
    speech_total = json.load(file)

len(speech_total)

828

In [3]:
# Setting up Groq API
import os
from dotenv import load_dotenv
from groq import Groq

load_dotenv()
client = Groq(api_key=os.getenv("GROQ_API"))

def get_groq_response(query):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        model="mixtral-8x7b-32768",
        temperature=0.5,
        max_tokens=1024,
    )
    return chat_completion.choices[0].message.content

In [4]:
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import faiss
import pickle
import torch

# Step 1: Load the embedding model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
embeddings = embed_model.encode(speech_total, show_progress_bar=True)
print(f"Embeddings shape: {embeddings.shape}")

# Step 2: Create and save FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype('float32'))

# Save the index and texts
faiss.write_index(index, "faiss_index.bin")
with open('original_texts.pkl', 'wb') as f:
    pickle.dump(speech_total, f)

# Step 3: Load the language model and tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set the pad token to be the end of sequence token
tokenizer.pad_token = tokenizer.eos_token

# Check if GPU is available, and if so set up device for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Create a text generation pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

# Create function to retrieve similar context from FAISS index based on query given
def retrieve_context(query, k=3):
    query_embedding = embed_model.encode([query])[0]
    D, I = index.search(np.array([query_embedding]).astype('float32'), k)
    retrieved_texts = [speech_total[i] for i in I[0]]
    return " ".join(retrieved_texts)

# Create function to generate a response, by combining retrieved context and response from Groq, then making it sound like Elon
def generate_response(query, max_new_tokens=200):
    # Get factual response from Groq
    groq_response = get_groq_response(query)
    
    # Retrieve context from your existing data
    context = retrieve_context(query)
    
    # Combine Groq response with retrieved context
    combined_input = f"Context: {context}\n\nGroq Response: {groq_response}\n\nQuery: {query}\n\nResponse:"
    
    # Tokenize the input
    inputs = tokenizer(combined_input, return_tensors="pt", padding=True, truncation=True, max_length=512)
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    
    # Generate Elon-style response
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_text = response.split("Response:")[-1].strip()
    
    # Call Elonify (function below) to make response sound like Elon
    return elonify(generated_text)

# Function to make response sound like Elon
def elonify(content):
    prompt = f"""
    Rewrite the following content in Elon Musk's speaking style, based on the training data:

    {content}

    Elon Musk's response:
    """
    
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=200,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    
    return tokenizer.decode(output[0], skip_special_tokens=True).split("Elon Musk's response:")[-1].strip()

/Users/brandonamarasingam/anaconda3/envs/pytorch/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/brandonamarasingam/anaconda3/envs/pytorch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

Embeddings shape: (828, 384)


/Users/brandonamarasingam/anaconda3/envs/pytorch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Query: What are Elon Musk's thoughts on AI?
Response: I am very happy with Elon's speech, but I do not want the media to go in and out of the room and say, 'Oh, they should have the power to do everything in their power.'
As we currently know, there are no 'natural' humans. Humans are natural, and when people are out there, we are responsible for their actions. Our natural behavior is different from ours. It is fundamentally different. We have an instinct for what is right and wrong. I have no doubt that human behavior will change over time. But as we know right now, humans have a very limited capacity to take action. If we can control our actions, then we will have more power. That is the key to our evolution. When we have not been able to control what we do, that will be a problem. One of those challenges is to understand that we don't have enough power, or we need to work with others to give us that power—or to use the


In [5]:
# Testing and Improving RAG System

import random # Used for sampling
from tqdm import tqdm # Provides progress bars

def evaluate_rag_system(num_samples=100):
    # Sample queries or create a test set
    test_queries = [
        "What does Elon Musk think about renewable energy?",
        "How does Elon Musk view the future of space exploration?",
        # Add more diverse queries here
    ]

    results = []
    for query in tqdm(random.sample(test_queries, min(num_samples, len(test_queries)))):
        response = generate_response(query)
        results.append({
            "query": query,
            "response": response,
            # You might add more metrics here, like response time, etc.
        })

    # Here you would typically add code to calculate metrics
    # such as relevance, coherence, factual accuracy, etc.
    # This often requires human evaluation or comparison against known ground truths

    return results

# Run evaluation
eval_results = evaluate_rag_system()
print(f"Evaluated {len(eval_results)} queries")
# Add code here to analyze and display results

100%|██████████| 2/2 [00:13<00:00,  6.75s/it]

Evaluated 2 queries


In [6]:
from IPython.display import clear_output

def interactive_qa():
    while True:
        query = input("Enter your question (or type 'exit' to quit): ")
        if query.lower() == 'exit':
            print("Thank you for using ElonAI, goodbye!")
            break
        
        if query.strip():
            print(f"\nQ: {query}")
            try:
                response = generate_response(query)
                print(f"\nA: {response}\n")
            except Exception as e:
                print(f"An error occurred: {str(e)}\n")
        else:
            print("Please enter a question.\n")

print("Type your question and press Enter to speak with.")
interactive_qa()

The Q&A system is ready. Type your question and press Enter.

Q: Can you explain what the nba is

A: I am a tremendous athlete. It has all the power to perform in your own way. My goal is to be a world-class athlete, and I will continue to strive to have that same ability. I hope you all enjoy this opportunity to learn more about how to develop and play for the NBA.
 
The next day, in an interview with the LA Times, Elon announced that he was taking the position of vice president of basketball operations. His decision to take the role of the General Manager of Basketball Operations after the start of his coaching career, was announced by the Los Angeles Times. That statement, made by that point, comes directly from Musk himself. He has made the announcement that his new position, as CEO of NBA Operations, is over. The decision is that all players are in the league, so no more than the ones that play on a regular basis. And that includes all people. So I'm going to give you a little mor